# NB1.5 — Streaming vs Batch (WebSockets & Polling)

En esta práctica implementamos **Streaming** con WebSockets y **Batch frecuente** con HTTP Polling. Guardaremos todo en **JSON Lines** en `../../data/` y compararemos latencia y continuidad.

**Objetivos**
- Entender diferencias clave **Batch vs Streaming**.
- Ejecutar un cliente **WebSocket** para ingesta continua.
- Ejecutar **HTTP Polling** como micro-batch.
- Comparar resultados en pandas.


## 1) Dependencias
Instalamos librerías requeridas para los scripts en `src/streaming/`.

In [1]:
!pip install -q websockets requests pandas

## 2) Streaming (WebSocket)

**Salida:** `../../data/stream_ws_YYYY-MM-DD.jsonl`

In [2]:
# Limita duración/eventos para la clase
!WS_MAX_EVENTS=150 WS_MAX_SECONDS=120 python ../../src/streaming/stream_dual_ws.py

"WS_MAX_EVENTS" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


## 3) Polling (HTTP)

**Salida:** `../../data/poll_bitcoin_YYYY-MM-DD.jsonl`

In [3]:
!python ../../src/streaming/poll_coincap_http.py

[POLL] 1/20 -> {'ts': '2025-10-05T18:04:05.016799+00:00', 'source': 'binance', 'instrument': 'BTCUSDT', 'price_usd': 123105.56}
[POLL] 2/20 -> {'ts': '2025-10-05T18:04:10.273627+00:00', 'source': 'binance', 'instrument': 'BTCUSDT', 'price_usd': 123105.57}
[POLL] 3/20 -> {'ts': '2025-10-05T18:04:15.550737+00:00', 'source': 'binance', 'instrument': 'BTCUSDT', 'price_usd': 123105.56}
[POLL] 4/20 -> {'ts': '2025-10-05T18:04:20.803869+00:00', 'source': 'binance', 'instrument': 'BTCUSDT', 'price_usd': 123105.57}
[POLL] 5/20 -> {'ts': '2025-10-05T18:04:26.051281+00:00', 'source': 'binance', 'instrument': 'BTCUSDT', 'price_usd': 123105.56}
[POLL] 6/20 -> {'ts': '2025-10-05T18:04:31.312153+00:00', 'source': 'binance', 'instrument': 'BTCUSDT', 'price_usd': 123105.56}
[POLL] 7/20 -> {'ts': '2025-10-05T18:04:36.559202+00:00', 'source': 'binance', 'instrument': 'BTCUSDT', 'price_usd': 123060.08}
[POLL] 8/20 -> {'ts': '2025-10-05T18:04:41.836118+00:00', 'source': 'binance', 'instrument': 'BTCUSDT', 

## 4) Cargar y comparar en pandas
Leemos ambos `.jsonl` y observamos diferencias de granularidad/latencia (aprox.).

In [4]:
from pathlib import Path
import pandas as pd

data_dir = Path("../../data/raw")

# STREAM
stream_file = sorted(data_dir.glob("stream_ws_*.jsonl"))[-1]
df_stream = pd.read_json(stream_file, lines=True)
print("STREAM:", stream_file)
display(df_stream.head())

# POLL 
poll_files = sorted(list(data_dir.glob("poll_binance_*.jsonl")) + list(data_dir.glob("poll_coincap_*.jsonl")))
poll_file = poll_files[-1]
df_poll = pd.read_json(poll_file, lines=True)
print("POLL:", poll_file)
display(df_poll.head())


IndexError: list index out of range

## 5) Reflexión (respuestas cortas)
**¿Quién tiene menor latencia?**  

**¿Qué pasa si el stream se cae?** 

**¿Cuál genera más duplicados o huecos temporales?**


1. El Streaming tiene menor latencia, pues se procesa casi en tiempo real. Mientras que el batch lo realiza por lotes generalmente programados.

2. Si el stream se cae se puede perder confiabilidad en los datos a menos que se tengan checkpoints o se hayan programado reintentos.

3. El Streaming al estar realizando un monitoreo en tiempo real genera más duplicados o huecos temporales, en cambio batch suele ser más consistente.